# Trabalho Prático 1  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Descrição e Abordagem ao Problema ##

O principal objetivo deste trabalho passa por criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver. Esta implementação em Python passa pelo uso dos módulos Cryptography, Ascon e Asyncio, cada um deles dedicado a uma determinada ação ocorrente na implementação. 

Usamos o pacote Ascon para autenticar o criptograma envolvido na counicação bem como os metadados envolventes. O Ascon é uma família de algoritmos de encriptação autenticada e de hashing concebidos para serem leves e fáceis de implementar, mesmo com contramedidas adicionais contra ataques de canais laterais. Foi concebido por uma equipa de criptógrafos da Universidade de Tecnologia de Graz, da Infineon Technologies e da Universidade de Radboud: Christoph Dobraunig, Maria Eichlseder, Florian Mendel e Martin Schläffer.

O NIST anunciou a seleção da família Ascon como o padrão de criptografia leve em fevereiro de 2023, após receber feedback público em um workshop. O NIST está trabalhando com a equipe Ascon para elaborar os padrões de criptografia leve.

Asyncio é uma biblioteca para escrever código concorrente usando a sintaxe async/await, por isso tiramos proveito dessa funcionalidade para poder implementar a comunicação cliente-servidor.

O pacote Cryptography é uma biblioteca Python que fornece fórmulas criptográficas e primitivas para programadores. O uso deste pacote serve para implementar uma AEAD com “Tweakable Block Ciphers”. 

AEAD (Authenticated Encryption with Associated Data) é um esquema de criptografia que simultaneamente assegura a confidencialidade dos dados (também conhecida como privacidade: a mensagem criptografada é impossível de entender sem o conhecimento de uma chave secreta) e a autenticidade (ou seja, é inalterável: a mensagem criptografada inclui uma etiqueta de autenticação que o remetente só pode calcular possuindo a chave secreta).

#### Imports ####

In [14]:
import os
import asyncio
import ascon

In [15]:

# from cryptography.hazmat.primitives import hashes
# from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
# from cryptography.hazmat.backends import default_backend
# # from cryptography.hazmat.primitives import serialization
# # from cryptography.hazmat.primitives.serialization import load_pem_public_key
# # from cryptography.hazmat.primitives.asymmetric import ec
# # from cryptography.hazmat.primitives.kdf.hkdf import HKDF

# import nest_asyncio
# nest_asyncio.apply()
# import random

# from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
# from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
# from cryptography.hazmat.primitives.ciphers.aead import AESGCM

from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from ascon import encrypt, decrypt, hash as ascon_hash

In [16]:
print(help(ascon))

Help on package ascon:

NAME
    ascon

PACKAGE CONTENTS
    _ascon

FUNCTIONS
    decrypt = ascon_decrypt(key, nonce, associateddata, ciphertext, variant='Ascon-128')
        Ascon decryption.
        key: a bytes object of size 16 (for Ascon-128, Ascon-128a; 128-bit security) or 20 (for Ascon-80pq; 128-bit security)
        nonce: a bytes object of size 16 (must not repeat for the same key!)
        associateddata: a bytes object of arbitrary length
        ciphertext: a bytes object of arbitrary length (also contains tag)
        variant: "Ascon-128", "Ascon-128a", or "Ascon-80pq" (specifies key size, rate and number of rounds)
        returns a bytes object containing the plaintext or None if verification fails
    
    encrypt = ascon_encrypt(key, nonce, associateddata, plaintext, variant='Ascon-128')
        Ascon encryption.
        key: a bytes object of size 16 (for Ascon-128, Ascon-128a; 128-bit security) or 20 (for Ascon-80pq; 128-bit security)
        nonce: a bytes object 

#### Implementação ####

In [17]:
# async def emitter(message, key, nonce):
#     # Gerar chave de cifra
#     cipher_key = key_derivation(key, nonce, "cipher")

#     # Criptografar a mensagem
#     ciphertext, tag = encrypt(message, cipher_key)

#     # Enviar a mensagem criptografada e o tag
#     return ciphertext, tag

# async def receiver(ciphertext, tag, key, nonce):
#     # Gerar chave de cifra
#     cipher_key = key_derivation(key, nonce, "cipher")

#     # Verificar a autenticidade do criptograma e dos metadados
#     if not verify_authenticity(ciphertext, tag, cipher_key):
#         return "Mensagem não autêntica!"

#     # Descriptografar a mensagem
#     decrypted_message = decrypt(ciphertext, cipher_key)

#     return decrypted_message

# def key_derivation(key, nonce, purpose):
#     # Simples chave de derivação usando PBKDF2HMAC
#     kdf = PBKDF2HMAC(
#         algorithm=hashes.SHA256(),
#         salt=nonce,
#         iterations=480000,
#         length=32
#     )
#     derived_key = kdf.derive(key + purpose.encode())
#     return derived_key

# def encrypt(message, key):
#     # Usando Ascon para criptografia simétrica
#     ascon = ascon_hash(key, variant="Ascon-Hash", hashlength=32)
#     ciphertext = ascon.encrypt(message.encode())

#     # Calcular o tag usando HMAC
#     h = hmac.HMAC(key, hashes.SHA256())
#     h.update(ciphertext)
#     tag = h.finalize()

#     return ciphertext, tag

# def verify_authenticity(ciphertext, tag, key):
#     # Verificar autenticidade usando HMAC
#     h = hmac.HMAC(key, hashes.SHA256())
#     h.update(ciphertext)
#     try:
#         h.verify(tag)
#         return True
#     except Exception as e:
#         return False

# def decrypt(ciphertext, key):
#     # Usar Ascon para descriptografar
#     ascon = ascon_hash(key, variant="Ascon-Hash", hashlength=32)
#     decrypted_message = ascon.decrypt(ciphertext)
#     return decrypted_message.decode()

# async def main():
#     # Chaves e nonces
#     emitter_key = b"emitter_key"
#     receiver_key = b"receiver_key"
#     emitter_nonce = b"emitter_nonce"
#     receiver_nonce = b"receiver_nonce"

#     # Mensagem para enviar
#     message_to_send = "Hello, receiver!"

#     # Comunicação assíncrona
#     ciphertext, tag =  emitter(message_to_send, emitter_key, emitter_nonce)
#     decrypted_message =  receiver(ciphertext, tag, receiver_key, receiver_nonce)

#     print("Mensagem original:", message_to_send)
#     print("Mensagem recebida:", decrypted_message)

# # Executar o loop de eventos assíncronos
# #await main()


In [18]:


async def key_derivation(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        iterations=100000,
        length=32,
        salt=salt,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())
    return key

async def generate_nonce():
    return os.urandom(16)

async def prg_key_generation():
    return os.urandom(16)  # Adjust the size based on the key size of your PRG

async def prg_generate(key, size):
    return ascon_hash(b"", variant="Ascon-Xof", hashlength=32)

async def encrypt_message(key, nonce, plaintext, associated_data=b""):
    ciphertext = encrypt(key, nonce, associated_data, plaintext, variant='Ascon-128')
    return ciphertext

async def decrypt_message(key, nonce, ciphertext, associated_data=b""):
    plaintext = decrypt(key, nonce, associated_data, ciphertext, variant='Ascon-128')
    return plaintext

async def communication(emitter_key, receiver_key):
    emitter_nonce = await generate_nonce()
    receiver_nonce = await generate_nonce()

    prg_emitter_key = await prg_key_generation()
    prg_receiver_key = await prg_key_generation()

    prg_emitter_nonce = await prg_generate(prg_emitter_key, 16)
    prg_receiver_nonce = await prg_generate(prg_receiver_key, 16)

    

    message = b"Hello, Receiver!"
    associated_data = b"Metadata"

    # Emitter encrypts the message
    ciphertext = await encrypt_message(emitter_key, emitter_nonce, message, associated_data)

    # Receiver decrypts the message
    decrypted_message = await decrypt_message(receiver_key, emitter_nonce, ciphertext, associated_data)

    print(f"Received message: {decrypted_message.decode()}")

async def main():
    password = "super_secret_password"
    salt = os.urandom(16)

    # Key derivation for Emitter and Receiver
    emitter_key = await key_derivation(password, salt)
    receiver_key = await key_derivation(password, salt)

    # Start the communication
    await communication(emitter_key, receiver_key)



In [19]:
# async def key_derivation(password, salt):
#     kdf = PBKDF2HMAC(
#         algorithm=hashes.SHA256(),
#         iterations=100000,
#         length=32,
#         salt=salt,
#         backend=default_backend()
#     )
#     key = kdf.derive(password.encode())
#     return key

# async def generate_nonce():
#     return os.urandom(16)

# async def encrypt(key, nonce, plaintext, associated_data=b""):
#     cipher = ascon.hash(key, variant="Ascon-Xof", hashlength=32)
#     ciphertext = cipher.encrypt(nonce, plaintext, associated_data)
#     return ciphertext

# async def decrypt(key, nonce, ciphertext, associated_data=b""):
#     cipher = ascon.hash(key, variant="Ascon-Xof", hashlength=32)
#     plaintext = cipher.decrypt(nonce, ciphertext, associated_data)
#     return plaintext

# async def communication(emitter_key, receiver_key):
#     emitter_nonce = await generate_nonce()
#     receiver_nonce = await generate_nonce()

#     message = b"Hello, Receiver!"
#     associated_data = b"Metadata"

#     # Emitter encrypts the message
#     ciphertext = await encrypt(emitter_key, emitter_nonce, message, associated_data)

#     # Receiver decrypts the message
#     decrypted_message = await decrypt(receiver_key, emitter_nonce, ciphertext, associated_data)

#     print(f"Received message: {decrypted_message.decode()}")

# async def main():
#     password = "super_secret_password"
#     salt = os.urandom(16)

#     # Key derivation for Emitter and Receiver
#     emitter_key = await key_derivation(password, salt)
#     receiver_key = await key_derivation(password, salt)

#     # Start the communication
#     await communication(emitter_key, receiver_key)

# if __name__ == "__main__":
#     asyncio.run(main())

In [20]:
# def keyGenerator():

#     # Gerar um par de chaves ECDH (priv e pub)

#     private_key = ec.generate_private_key(ec.SECP256K1)
#     public_key = private_key.public_key().public_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PublicFormat.SubjectPublicKeyInfo
#     )
    
#     return private_key, public_key

In [21]:
# def sigGenerator(public_key):

#     # gerar chaves ecdsa

#     priv_key_ecdsa = ec.generate_private_key(ec.SECP384R1)

#     public_key_ecdsa = priv_key_ecdsa.public_key().public_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
#     digest = hashes.Hash(hashes.SHA256(public_key).digest())
    
#     sig = priv_key_ecdsa.sign(digest)

#     return public_key_ecdsa, sig

In [22]:
# def sharedkeysGenerator(public_key, private_key):

#     #Gerar chaves partilhas a partir da chave publica recebida
    
#     shared_key = private_key.exchange(public_key)
    
#     derived_key = HKDF(            
#             algorithm=hashes.SHA256(),
#             length=64,

#             salt=None,
#             info=b'handshake data'
#         ).derive(shared_key)
    
#     # Dividir em chave para cifrar/decifrar e chave de autenticação
    
#     cipher_key = derived_key[:32]
#     mac_key = derived_key[32:]

#     return cipher_key, mac_key

In [23]:
# def cifragem(msg, metadados, chiper_key, mac_key): 

#     msg = msg.encode('utf8') # conversão do texto limpo para bytes
    
#     # Gerar o Nonce
    
#     digest = hashes.Hash(hashes.SHA256()) 
#     nonce = digest.finalize()

#     # Cifragem

#     cipher = Cipher(algorithms.AES(chiper_key), 
#                     modes.GCM(nonce), 
#                     backend=default_backend())
#     encryptor = cipher.encryptor()
#     ciphertext = encryptor.update(msg) + encryptor.finalize()
#     tag = encryptor.tag

#     encryptor.authenticate_additional_data(metadados) 

#     # Gerar tag HMAC

#     tag_ascon = ascon.ascon_hash(mac_key, msg, metadados) 

#     # Dicionário com os dadosa necessários para fazer a decifragem

#     dados = {'ct':ciphertext, 'metadados': metadados, 'nonce': nonce, 'ascon' : tag_ascon}

#     return dados

In [24]:
# def asconGenerator(mac_key, msg, metadados):
    
#     #Gerar tag hmac

#     tag = ascon.ascon_hash(mac_key, hashes.SHA256(), backend=default_backend())

#     tag.update(msg + metadados)

#     return tag.finalize()

In [25]:
# def decifragem(ciphered_text, metadados, key_mac, key_cipher, tag, nonce):

#     # Verificar o código de autenticação

#     tag_ascon = ascon.ascon_hash(key_mac, hashes.SHA256(), backend = default_backend())
#     tag_ascon.update(ciphered_text + metadados)
#     tag_ascon = tag_ascon.finalize()

#     if(tag_ascon != tag_ascon):
#         return None
    
#     # Decifragem

#     decryptor = Cipher(algorithms.AES(key_cipher),
#                        modes.GCM(nonce, tag_ascon),
#                        backend=default_backend()).decryptor()
    
#     plain_text = decryptor.update(ciphered_text) + decryptor.finalize()


#     return plain_text.decode('utf-8')

In [26]:

# async def emitter(queue, msg):

#     # Gerar chaves ECDH

#     E_priv_key, E_pub_key = keyGenerator()

#     sig, pub_key_ECDSA= sigGenerator(E_pub_key)


#     # Enviar a chave publica

#     await queue.put(E_pub_key)

#     await asyncio.sleep(random.random()) 

#     await queue.put(pub_key_ECDSA)

#     await asyncio.sleep(random.random()) 

#     await queue.put(sig)

#     await asyncio.sleep(random.random()) 

#     # Receber chave publica ECDH, ECDSA e sig

#     R_pub_key = await queue.get()

#     await asyncio.sleep(random.random())

#     R_pub_key_ECDSA = await queue.get()

#     await asyncio.sleep(random.random())

#     sig = await queue.get()

#     await asyncio.sleep(random.random())

    

#     hash = hashes.Hash(hashes.SHA256(sig).digest())

#     try:
        
#         # Verificação da Assinatura

#         R_pub_key_ECDSA.verify(sig,
#                                    hash,
#                                    ec.ECDSA(hashes.SHA256())
#         )

#         print("Assinatura válida!")


#         # Criar chaves partilhadas (cipher e mac keys)

#         chiper_key, mac_key = sharedkeysGenerator(R_pub_key, E_priv_key)

#         # Cifragem

#         metadados = os.urandom(16)

#         cifragem = cifragem(msg, metadados, chiper_key, mac_key)

#         # Enviar o resultado da cifragem para o Receiver

#         await queue.put(cifragem)

#         await asyncio.sleep(random.random())

#     except:
#         print("Assinatura Inválida!")


#     # indicate the producer is done
#     await queue.put(None)


# async def receiver(queue):
    
#     # Gerar chaves pública e privada ECDH, publica ECDSA e sig (Receiver)

#     R_priv_key, R_pub_key = keyGenerator()

#     sig, R_pub_key = sigGenerator(R_pub_key)

#     # Receber chave publica ECDH, ECDSA e sig (Emitter)
  
#     E_pub_key = await queue.get()

#     await asyncio.sleep(random.random())     
#     E_pub_key_ECDSA = await queue.get()

#     await asyncio.sleep(random.random())    
#     sig = await queue.get()

#     print('Receiver: Recebi chave pública do Emitter')
#     print(E_pub_key)

#     print('Receiver : Recebi a chave da assinatura')
#     print(E_pub_key_ECDSA)

#     print('Receiver : Recebi a assinatura')

#     # Enviar chave publica ECDH, ECDSA e sig (Receiver)
    
#     await queue.put(R_pub_key)
#     print('Receiver: Enviei chave pub')

#     await asyncio.sleep(random.random()) 

#     await queue.put(R_pub_key)
#     print('Receiver: Enviei chave pub ECDSA')
#     await asyncio.sleep(random.random())

#     await queue.put(sig)
#     print('Receiver: Enviei sig')

#     hash = hashes.Hash(hashes.SHA256(sig).digest())

#     try:
        
#         # Verificação da Assinatura

#         E_pub_key_ECDSA.verify(sig,
#                                    hash,
#                                    ec.ECDSA(hashes.SHA256())
#         )

#         print("Assinatura válida!")

#         chiper_key, mac_key = sharedkeysGenerator(R_pub_key, R_priv_key)

#         #receber cifragem

#         await asyncio.sleep(random.random())   
#         cifragem = await queue.get()
    
#         ciphertext = cifragem['ct']
#         nonce = cifragem['nonce']
#         hmac = cifragem['tag']
#         metadados = cifragem['metadados']

#         # Decifragem

#         plaintext = decifragem(ciphertext, metadados, mac_key, chiper_key, ascon, nonce)

#         print (plaintext)


#     except:
#         print("Assinatura Inválida!")
    




# if __name__ == '__main__':
#     msg = input("Message:")
#     loop = asyncio.get_event_loop()
#     queue = asyncio.Queue(loop=loop)
#     asyncio.ensure_future(emitter(queue, msg), loop=loop)
#     loop.run_until_complete(receiver(queue))